In [ ]:
import os
import pandas as pd

# Step 1: Read data from ASCII files
folder_path = "/Users/soumilhooda/Desktop/WD/Data-WD/Weather/ALL-VALID"
data = {}

for file_name in os.listdir(folder_path):
    if file_name.startswith("rainnn") or file_name.startswith("maxtmp") or file_name.startswith("mintmp"):
        year = file_name[-10:-6]
        data_type = file_name[:6]
        df = pd.read_csv(os.path.join(folder_path, file_name), delim_whitespace=True, header=None)
        num_days = df.shape[1] - 2  # Deducting 2 for Latitude and Longitude columns
        date_range = pd.date_range(start=f'{year}-01-01', periods=num_days)
        date_columns = date_range.strftime('%Y-%m-%d').tolist()
        df.columns = ['Latitude', 'Longitude'] + date_columns
        data[(data_type, year)] = df
        
# Step 2: Calculate average temperature
for year in range(1951, 2024):
    maxtmp = data[('maxtmp', str(year))]
    mintmp = data[('mintmp', str(year))]
    avg_tmp = (maxtmp.iloc[:, 2:] + mintmp.iloc[:, 2:]) / 2
    data[('avgtmp', str(year))] = pd.concat([maxtmp.iloc[:, :2], avg_tmp], axis=1)


# Step 3: Create dictionary with (latitude, longitude) pairs and rainfall/temperature data
result = {}

for year in range(1951, 2024):
    for lat, lon in zip(data[('rainnn', str(year))]['Latitude'], data[('rainnn', str(year))]['Longitude']):
        if (lat, lon) not in result:
            result[(lat, lon)] = pd.DataFrame(columns=['Rainfall', 'Temperature'])
        rain = data[('rainnn', str(year))][(data[('rainnn', str(year))]['Latitude'] == lat) & (data[('rainnn', str(year))]['Longitude'] == lon)].iloc[:, 2:].values.flatten()
        temp = data[('avgtmp', str(year))][(data[('avgtmp', str(year))]['Latitude'] == lat) & (data[('avgtmp', str(year))]['Longitude'] == lon)].iloc[:, 2:].values.flatten()
        date_index = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31')
        df = pd.DataFrame({'Rainfall': rain, 'Temperature': temp}, index=pd.DatetimeIndex(date_index))
        result[(lat, lon)] = pd.concat([result[(lat, lon)], df])

# Filter the result dictionary to keep only key-value pairs with 26663 rows
filtered_result = {key: value for key, value in result.items() if len(value) == 26663}

# Load GeoLocations.csv
geo_df = pd.read_csv("/Users/soumilhooda/Desktop/WD/Data-WD/GeoLocations/GeoLocations.csv")

data = {}

for key, df in filtered_result.items():
    lat, lon = key
    state_df = geo_df[(geo_df['Latitude'] == lat) & (geo_df['Longitude'] == lon)]
    if not state_df.empty:
        state_name = state_df['State Name'].iloc[0]
        # Check if the state name is not 'MAHARASHTRA'
        if state_name != 'MAHARASHTRA':
            new_key = (lat, lon, state_name)
            data[new_key] = df

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import AutoReg
import datetime as dt
from scipy import interpolate

def T_model(x, a, b, c, d, alpha, beta, theta):
    omega = 2 * np.pi / 365.25
    return a + b*x + c*x**2 + d*x**3 + alpha*np.sin(omega*x + theta) + beta*np.cos(omega*x + theta)

def dT_model(x, a, b, c, d, alpha, beta, theta):
    omega = 2 * np.pi / 365.25
    dT = b + 2*c*x + 3*d*x**2 + alpha*omega*np.cos(omega*x + theta) - beta*omega*np.sin(omega*x + theta)
    return dT

def T_error_func(t, temp, params):
    return temp - T_model(t, *params)

def T_fit_and_visualize(latlon_data):
    # Calculate time dimension for the entire dataset
    t = (latlon_data.index - latlon_data.index[0]).days.values
    T = latlon_data['Temperature'].values

    # Split data into train and test sets
    train_data, test_data = train_test_split(latlon_data, test_size=0.04, shuffle=False, random_state=42)
    t_train = (train_data.index - train_data.index[0]).days.values
    temp_train = train_data['Temperature'].values
    t_test = (test_data.index - test_data.index[0]).days.values
    temp_test = test_data['Temperature'].values

    # Fit the model to the train data
    initial_guess = [0, 0, 0, 0, 0, 0, 0]  # Initial guess for parameters
    params, _ = curve_fit(T_model, t_train, temp_train, p0=initial_guess)

    # Fit an AutoRegressive model to residuals
    residuals_all = T - T_model(t, *params)
    residuals_all_df = pd.DataFrame(data=residuals_all, index=latlon_data.index)
    residuals_all_df.index = pd.DatetimeIndex(residuals_all_df.index).to_period('D')
    model = AutoReg(residuals_all_df.squeeze(), lags=1, old_names=True, trend='n')
    model_fit = model.fit()
    
    # Calculate kappa
    gamma = model_fit.params[0]
    kappa = 1 - gamma

    # Store parameters for T model equation
    Tbar_params = params

    temp_t = latlon_data.copy()  
    if isinstance(temp_t.index, pd.DatetimeIndex):
        first_ord = temp_t.index.map(dt.datetime.toordinal)[0]
        temp_t.index = temp_t.index.map(dt.datetime.toordinal)

    temp_t['model_fit'] = T_model(temp_t.index - first_ord, *Tbar_params)

    if not isinstance(temp_t.index, pd.DatetimeIndex):
        temp_t.index = temp_t.index.map(dt.datetime.fromordinal)

    return T_model, dT_model, Tbar_params, kappa

T_models = {}
dT_models = {}
Tbar_params_list = {}
kappas = {}

for lat, lon, state in data:
    latlonstate_data = data[(lat, lon, state)]
    T_model_func, dT_model_func, Tbar_params, kappa = T_fit_and_visualize(latlonstate_data)
    T_models[(lat, lon, state)] = T_model_func
    dT_models[(lat, lon, state)] = dT_model_func
    Tbar_params_list[(lat, lon, state)] = Tbar_params
    kappas[(lat, lon, state)] = kappa

In [ ]:
referencetemperature = {
    'PUNJAB_HDD': 16.46375,
    'PUNJAB_CDD': 27.778750000000002,
    'GUJARAT_HDD': 23.486538461538462,
    'GUJARAT_CDD': 29.288846153846155,
    'HARYANA_HDD': 13.93,
    'HARYANA_CDD': 29.15833333333333,
    'HIMACHAL PRADESH_HDD': 12.38,
    'HIMACHAL PRADESH_CDD': 24.453333333333333,
    'MADHYA PRADESH_HDD': 19.918513513513517,
    'MADHYA PRADESH_CDD': 27.11621621621622,
    'BIHAR_HDD': 19.80333333333333,
    'BIHAR_CDD': 25.22666666666667,
    'UTTAR PRADESH_HDD': 14.94980769230769,
    'UTTAR PRADESH_CDD': 28.63653846153846,
    'KARNATAKA_HDD': 23.28708333333333,
    'KARNATAKA_CDD': 25.574583333333337,
    'TELANGANA_HDD': 21.489444444444445,
    'TELANGANA_CDD': 27.185000000000002,
    'ANDHRA PRADESH_HDD': 23.011666666666667,
    'ANDHRA PRADESH_CDD': 27.52333333333333,
    'RAJASTHAN_HDD': 15.723148148148145,
    'RAJASTHAN_CDD': 25.690740740740743,
    'ORISSA_HDD': 19.625714285714285,
    'ORISSA_CDD': 28.180714285714288
}

tref = {}

for key, value in referencetemperature.items():
    city, feature = key.split('_')
    tref[(feature, city)] = value

WINTER_MONTHS = [1, 2, 10, 11, 12]
MONSOON_MONTHS = [3, 4, 5, 6, 7, 8, 9]

In [ ]:
import numpy as np
from scipy import interpolate
import pandas as pd

# Initialize an empty dictionary to store volatility for each city
volatility = {}

for key in data:
    latlonstate_data = data[key]
    temp_vol = latlonstate_data['Temperature'].copy(deep=True)
    temp_vol = temp_vol.to_frame()
    temp_vol['day'] = temp_vol.index.dayofyear

    vol = temp_vol.groupby(['day'])['Temperature'].agg(['mean', 'std'])
    days = np.array(vol['std'].index)
    T_std = np.array(vol['std'].values)

    def spline(knots, x, y):
        x_new = np.linspace(0, 1, knots+2)[1:-1]
        t, c, k = interpolate.splrep(x, y, t=np.quantile(x, x_new), s=3)
        yfit = interpolate.BSpline(t,c, k)(x)
        return yfit

    volatility[key] = spline(15, days, T_std)

def euler_step(row, kappa, M, lamda):
    """Function for euler scheme approximation step in
    modified OH dynamics for temperature simulations
    Inputs:
    - dataframe row with columns: T, Tbar, dTbar and vol
    - kappa: rate of mean reversion
    Output:
    - temp: simulated next day temperatures
    """
    if row['T'] != np.nan:
        T_i = row['Tbar']
    else:
        T_i = row['T']
    T_det = T_i + row['dTbar']
    T_mrev =  kappa*(row['Tbar'] - T_i)
    sigma = row['vol']*np.random.randn(M)
    riskn = lamda*row['vol']
    return T_det + T_mrev + sigma - riskn

def monte_carlo_temp(trading_dates, Tbar_params, vol_model, first_ord, kappa, key, no_sims, lamda=0):
    """Monte Carlo simulation of temperature
    Inputs:
    - trading_dates: pandas DatetimeIndex from start to end dates
    - M: number of simulations
    - Tbar_params: parameters used for Tbar model
    - vol_model: fitted volatility model with days in year index
    - first_ord: first ordinal of fitted Tbar model
    Outputs:
    - mc_temps: DataFrame of all components and simulated temperatures
    """
    # print('This is the city inside monte carlo function: ', city)
    kappa=kappas[key]
    # print('This is the kappa value inside monte carlo function: ', kappa)

    if isinstance(trading_dates, pd.DatetimeIndex):
        trading_date=trading_dates.map(dt.datetime.toordinal)

    Tbar_params = Tbar_params[key]
    vol_model = vol_model[key]
    first_ord = first_ord[key]

    Tbars = T_model(trading_date-first_ord, *Tbar_params)
    dTbars = dT_model(trading_date-first_ord, *Tbar_params)
    mc_temps = pd.DataFrame(data=np.array([Tbars, dTbars]).T,
                            index=trading_dates, columns=['Tbar','dTbar'])
    mc_temps['day'] = mc_temps.index.dayofyear
    mc_temps['vol'] = vol_model[mc_temps['day']-1]

    mc_temps['T'] = mc_temps['Tbar'].shift(1)
    data = mc_temps.apply(euler_step, args=[kappa, no_sims, lamda], axis=1)
    mc_sims = pd.DataFrame(data=[x for x in [y for y in data.values]],
                 index=trading_dates,columns=range(1,no_sims+1))

    return mc_temps, mc_sims

data_start_date = dt.datetime(1951, 1, 1)

no_sims=1000

def determine_option_type(start_date, end_date):
    """Determines the option type based on contract dates."""
    start_month = dt.datetime.strptime(start_date, "%Y-%m-%d").month
    end_month = dt.datetime.strptime(end_date, "%Y-%m-%d").month

    if start_month in WINTER_MONTHS and end_month in WINTER_MONTHS:
        return 'winter'
    elif start_month in MONSOON_MONTHS and end_month in MONSOON_MONTHS:
        return 'monsoon'
    else:
        return None 


# Define first ordinal values for each city
first_ord = {}
for key in data:
    first_ord[key] = data_start_date.toordinal()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def temperature_option(trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='c', no_sims=1000, lamda=0):
    "Evaluates the price of a temperature call option"
    for city in Tbar_params_list.keys():
        mc_temps, mc_sims = monte_carlo_temp(trading_dates, Tbar_params_list, volatility, first_ord, kappas, city, no_sims=1000, lamda=0)
        N, M = np.shape(mc_sims)
        mc_arr = mc_sims.values
        # print('this is mc_arr value inside temp option after exiting monte carlo: ', mc_arr)
        if option_type == 'winter':
            DD = np.sum(np.maximum(tref['HDD',city[2]]-mc_arr,0), axis=0)
        elif option_type == 'monsoon':
            DD = np.sum(np.maximum(mc_arr-tref['CDD',city[2]],0), axis=0)
        else:
            print('Options cannot run across or outside a season.')
            exit
        if opt == 'c':
            CT = alpha*np.maximum(DD-K,0)
        else:
            CT = alpha*np.maximum(K-DD,0)
        C0 = np.exp(-r*tau)*np.sum(CT)/M
        sigma = np.sqrt( np.sum( (np.exp(-r*tau)*CT - C0)**2) / (M-1) )
        SE = sigma/np.sqrt(M)
        # print(f'{opt.upper()} Price for {city} City: {np.round(C0, 2)} +/- {np.round(SE*2, 2)} (2SE)')
        return C0


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt  # For date calculations

# Function to calculate years between two dates
def years_between(d1, d2):
    d1 = dt.datetime.strptime(d1, "%Y-%m-%d")
    d2 = dt.datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days) / 365.25

monsoon_strike_prices = np.arange(110, 130, 5)
monsoon_start = "2024-05-05"
monsoon_end = "2024-07-20"
tau = years_between(monsoon_start, monsoon_end)
option_type = determine_option_type(monsoon_start, monsoon_end)
r = 0.05
alpha = 25

# Function to plot call and put prices for a given city and season
def plot_options(city, season, strike_prices, base_value, trading_dates):
    call_prices = []
    put_prices = []

    for K in strike_prices:
        # Calculate call and put prices using updated trading_dates
        call_price = temperature_option(trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='c')
        put_price = temperature_option(trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='p')

        call_prices.append(call_price)
        put_prices.append(put_price)

    return call_prices, put_prices

# Iterate over states
for state in set(city_data[2] for city_data in Tbar_params_list):
    fig, ax = plt.subplots(figsize=(12, 5))

    # Get cities for the current state
    cities = [city for city in Tbar_params_list.keys() if city[2] == state]

    if cities:
        first_city = cities[0]

        monsoon_trading_dates = pd.date_range(start=monsoon_start, end=monsoon_end, freq='D')

        # Calculate and plot option prices for the first city (Monsoon)
        call_prices_first, put_prices_first = plot_options(first_city, "monsoon", monsoon_strike_prices, None, monsoon_trading_dates)
        ax.plot(monsoon_strike_prices, call_prices_first, label=f'{first_city} - Call Option', color='blue')
        ax.plot(monsoon_strike_prices, put_prices_first, label=f'{first_city} - Put Option', color='orange')

        # Calculate variances for all other cities
        variances = []
        for city in cities[1:]:
            call_prices, put_prices = plot_options(city, "monsoon", monsoon_strike_prices, None, monsoon_trading_dates)
            variance = np.sum((np.array(call_prices) - np.array(call_prices_first)) ** 2) + np.sum((np.array(put_prices) - np.array(put_prices_first)) ** 2)
            variances.append((city, variance))

        # Get the city with the maximum variance
        max_variance_city = sorted(variances, key=lambda x: x[1], reverse=True)[:2]

        # Plot variance for the city with maximum variance
        for city, variance in max_variance_city:
            call_prices, put_prices = plot_options(city, "monsoon", monsoon_strike_prices, None, monsoon_trading_dates)
            ax.fill_between(monsoon_strike_prices, call_prices, call_prices_first, label=f'{city} - Call Option Variance', alpha=0.2)
            ax.fill_between(monsoon_strike_prices, put_prices, put_prices_first, label=f'{city} - Put Option Variance', alpha=0.2)

        # Set plot labels and formatting
        ax.set_xlabel('Strike Price', weight='bold')
        ax.set_ylabel('Option Price (USD)', weight='bold')
        ax.set_title(f'Monsoon Option Prices Comparison for {state.title()}', weight='bold')  # Title added here
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
        ax.set_xticks(np.arange(min(monsoon_strike_prices), max(monsoon_strike_prices) + 1, 1))
        ax.set_yticks(np.arange(0, max(call_prices_first + put_prices_first) + 1, 15))

        ax.tick_params(axis='both', which='both')
        ax.text(0.05, 0.95, state, transform=ax.transAxes, weight='bold', fontsize=12, verticalalignment='top')  # Add state name
        plt.tight_layout()
        plt.show()
    else:
        print(f"No cities found for state: {state}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt  # For date calculations

# Function to calculate years between two dates
def years_between(d1, d2):
    d1 = dt.datetime.strptime(d1, "%Y-%m-%d")
    d2 = dt.datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days) / 365.25

winter_strike_prices = np.arange(45, 55, 1)
winter_start = "2024-10-05"
winter_end = "2025-02-20"
tau = years_between(winter_start, winter_end)
option_type = determine_option_type(winter_start, winter_end)
r = 0.05
alpha = 25

# Function to plot call and put prices for a given city and season
def plot_options(city, season, strike_prices, base_value, trading_dates):
    call_prices = []
    put_prices = []

    for K in strike_prices:
        # Calculate call and put prices using updated trading_dates
        call_price = temperature_option(trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='c')
        put_price = temperature_option(trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='p')

        call_prices.append(call_price)
        put_prices.append(put_price)

    return call_prices, put_prices

# Iterate over states
for state in set(city_data[2] for city_data in Tbar_params_list):
    fig, ax = plt.subplots(figsize=(12, 5))

    # Get cities for the current state
    cities = [city for city in Tbar_params_list.keys() if city[2] == state]

    if cities:
        first_city = cities[0]

        # Create trading_dates for winter and monsoon
        winter_trading_dates = pd.date_range(start=winter_start, end=winter_end, freq='D')

        # Calculate and plot option prices for the first city (Monsoon)
        call_prices_first, put_prices_first = plot_options(first_city, "winter", winter_strike_prices, None, winter_trading_dates)
        ax.plot(winter_strike_prices, call_prices_first, label=f'{first_city} - Call Option', color='blue')
        ax.plot(winter_strike_prices, put_prices_first, label=f'{first_city} - Put Option', color='orange')

        # Calculate variances for all other cities
        variances = []
        for city in cities[1:]:
            call_prices, put_prices = plot_options(city, "winter", winter_strike_prices, None, winter_trading_dates)
            variance = np.sum((np.array(call_prices) - np.array(call_prices_first)) ** 2) + np.sum((np.array(put_prices) - np.array(put_prices_first)) ** 2)
            variances.append((city, variance))

        # Get the city with the maximum variance
        max_variance_city = sorted(variances, key=lambda x: x[1], reverse=True)[:2]

        # Plot variance for the city with maximum variance
        for city, variance in max_variance_city:
            call_prices, put_prices = plot_options(city, "winter", winter_strike_prices, None, winter_trading_dates)
            ax.fill_between(winter_strike_prices, call_prices, call_prices_first, label=f'{city} - Call Option Variance', alpha=0.2)
            ax.fill_between(winter_strike_prices, put_prices, put_prices_first, label=f'{city} - Put Option Variance', alpha=0.2)

        # Set plot labels and formatting
        ax.set_xlabel('Strike Price', weight='bold')
        ax.set_ylabel('Option Price (USD)', weight='bold')
        ax.set_title(f'Winter Option Prices Comparison for {state.title()}', weight='bold')  # Title added here
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
        ax.set_xticks(np.arange(min(winter_strike_prices), max(winter_strike_prices) + 1, 1))
        ax.set_yticks(np.arange(0, max(call_prices_first + put_prices_first) + 1, 20))

        ax.tick_params(axis='both', which='both')
        ax.text(0.05, 0.95, state, transform=ax.transAxes, weight='bold', fontsize=12, verticalalignment='top')  # Add state name
        plt.tight_layout()
        plt.show()
    else:
        print(f"No cities found for state: {state}")


In [ ]:
def scenario_analysis(trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='c', no_sims=1000, lamda_scenarios=[0, 0.1, 0.2]):
    """Performs scenario analysis for different lambda values."""
    call_prices_scenarios = {}
    put_prices_scenarios = {}

    for lamda in lamda_scenarios:
        call_prices = []
        put_prices = []

        for city in Tbar_params_list.keys():
            mc_temps, mc_sims = monte_carlo_temp(trading_dates, Tbar_params_list, volatility, first_ord, kappas, city, no_sims, lamda)
            N, M = np.shape(mc_sims)
            mc_arr = mc_sims.values

            if option_type == 'winter':
                DD = np.sum(np.maximum(tref['HDD', city[2]] - mc_arr, 0), axis=0)
            elif option_type == 'monsoon':
                DD = np.sum(np.maximum(mc_arr - tref['CDD', city[2]], 0), axis=0)
            else:
                print('Options cannot run across or outside a season.')
                exit

            if opt == 'c':
                CT = alpha * np.maximum(DD - K, 0)
            else:
                CT = alpha * np.maximum(K - DD, 0)
            C0 = np.exp(-r * tau) * np.sum(CT) / M
            call_prices.append(C0)
            put_prices.append(C0)  # Assuming put-call parity holds

        call_prices_scenarios[lamda] = call_prices
        put_prices_scenarios[lamda] = put_prices

    return call_prices_scenarios, put_prices_scenarios

def plot_call_scenario_analysis(call_prices_scenarios, state_names, lamda_scenarios):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot for call prices
    for state_index, state_name in enumerate(state_names):
        call_prices = [call_prices_scenarios[lamda][state_index] for lamda in lamda_scenarios]
        ax.plot(lamda_scenarios, call_prices, label=f'{state_name} - Call Option', marker='o')

    ax.set_xlabel('Lambda', weight='bold')
    ax.set_ylabel('Call Option Price (USD)', weight='bold')
    ax.set_title('Call Option Risk Aversion Scenario Analysis for Monsoon Months', weight='bold')
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.tight_layout()
    plt.show()

monsoon_start = "2024-03-05"
monsoon_end = "2024-09-20"
monsoon_trading_dates = pd.date_range(start=monsoon_start, end=monsoon_end, freq='D')
tau = years_between(monsoon_start, monsoon_end)
option_type = determine_option_type(monsoon_start, monsoon_end)
K = 110
r = 0.05
alpha = 25
lamda_scenarios = [0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2] 

city_indices = [68, 125]  
state_names = ['Telanagana', 'Orissa']
call_prices_scenarios, _ = scenario_analysis(monsoon_trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='c', no_sims=1000, lamda_scenarios=lamda_scenarios)

plot_call_scenario_analysis(call_prices_scenarios, state_names, lamda_scenarios)


In [ ]:
def scenario_analysis(trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='c', no_sims=1000, lamda_scenarios=[0, 0.1, 0.2]):
    """Performs scenario analysis for different lambda values."""
    call_prices_scenarios = {}
    put_prices_scenarios = {}

    for lamda in lamda_scenarios:
        call_prices = []
        put_prices = []

        for city in Tbar_params_list.keys():
            mc_temps, mc_sims = monte_carlo_temp(trading_dates, Tbar_params_list, volatility, first_ord, kappas, city, no_sims, lamda)
            N, M = np.shape(mc_sims)
            mc_arr = mc_sims.values

            if option_type == 'winter':
                DD = np.sum(np.maximum(tref['HDD', city[2]] - mc_arr, 0), axis=0)
            elif option_type == 'monsoon':
                DD = np.sum(np.maximum(mc_arr - tref['CDD', city[2]], 0), axis=0)
            else:
                print('Options cannot run across or outside a season.')
                exit

            if opt == 'c':
                CT = alpha * np.maximum(DD - K, 0)
            else:
                CT = alpha * np.maximum(K - DD, 0)
            C0 = np.exp(-r * tau) * np.sum(CT) / M
            call_prices.append(C0)
            put_prices.append(C0)  # Assuming put-call parity holds

        call_prices_scenarios[lamda] = call_prices
        put_prices_scenarios[lamda] = put_prices

    return call_prices_scenarios, put_prices_scenarios

def plot_call_scenario_analysis(call_prices_scenarios, state_names, lamda_scenarios):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot for call prices
    for state_index, state_name in enumerate(state_names):
        call_prices = [call_prices_scenarios[lamda][state_index] for lamda in lamda_scenarios]
        ax.plot(lamda_scenarios, call_prices, label=f'{state_name} - Call Option', marker='o')

    ax.set_xlabel('Lambda', weight='bold')
    ax.set_ylabel('Call Option Price (USD)', weight='bold')
    ax.set_title('Call Option Risk Aversion Scenario Analysis for Winter Months', weight='bold')
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.tight_layout()
    plt.show()

winter_start = "2024-10-05"
winter_end = "2025-02-20"
tau = years_between(winter_start, winter_end)
winter_trading_dates = pd.date_range(start=winter_start, end=winter_end, freq='D')
option_type = determine_option_type(winter_start, winter_end)
K = 50
r = 0.05
alpha = 25
lamda_scenarios = [0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2] 

city_indices = [68, 125]  
state_names = ['Telanagana', 'Orissa']
call_prices_scenarios, _ = scenario_analysis(winter_trading_dates, Tbar_params_list, volatility, kappas, r, alpha, K, tau, first_ord, option_type, opt='c', no_sims=1000, lamda_scenarios=lamda_scenarios)

plot_call_scenario_analysis(call_prices_scenarios, state_names, lamda_scenarios)
